# Auth Endpoint Tests

This notebook contains all commands and code to test the authentication endpoints on `dev.krija.info:8000`.

## Test Credentials
- **Email**: krijajannis@gmail.com
- **Password**: 221224

## Server Info
- **Base URL**: http://dev.krija.info:8000/api
- **Debug Mode**: True (rate limiting should be disabled)

⚠️ **Note**: If you get 429 (rate limit) errors, wait 15 minutes or restart the server.

## Individual Endpoint Tests

Run tests for specific endpoints:

In [ ]:
# Test registration endpoint
cd /Users/jannis/Developer/Cookify/backend
python -m pytest tests/auth/test_register.py -v

In [ ]:
# Test token refresh endpoint
cd /Users/jannis/Developer/Cookify/backend
python -m pytest tests/auth/test_refresh.py -v

In [ ]:
# Test logout endpoint
cd /Users/jannis/Developer/Cookify/backend
python -m pytest tests/auth/test_logout.py -v

In [ ]:
# Test user info endpoints
cd /Users/jannis/Developer/Cookify/backend
python -m pytest tests/auth/test_me.py -v
python -m pytest tests/auth/test_profile.py -v
python -m pytest tests/auth/test_update_profile.py -v

In [ ]:
# Test password endpoints
cd /Users/jannis/Developer/Cookify/backend
python -m pytest tests/auth/test_forgot_password.py -v
python -m pytest tests/auth/test_reset_password.py -v
python -m pytest tests/auth/test_change_password.py -v
python -m pytest tests/auth/test_password_strength.py -v

In [ ]:
# Test email verification endpoints
cd /Users/jannis/Developer/Cookify/backend
python -m pytest tests/auth/test_verify_email.py -v
python -m pytest tests/auth/test_resend_verification.py -v

In [ ]:
# Test utility endpoints
cd /Users/jannis/Developer/Cookify/backend
python -m pytest tests/auth/test_health.py -v
python -m pytest tests/auth/test_user_info.py -v
python -m pytest tests/auth/test_dev_login.py -v

## Manual Testing (Interactive)

Use this when rate limiting is active or you want to test manually:

In [ ]:
# Run manual test script
cd /Users/jannis/Developer/Cookify/backend
python tests/auth/manual_test.py

## Interactive Testing with Python

Test endpoints directly from Python:

In [ ]:
import asyncio
import httpx
import json
from typing import Dict, Any

# Server configuration
BASE_URL = "http://dev.krija.info:8000/api"
TEST_CREDENTIALS = {
    "email": "krijajannis@gmail.com",
    "password": "221224"
}

print("🚀 Setup complete! Ready for interactive testing.")

In [ ]:
async def test_endpoint(method: str, endpoint: str, data: Dict[str, Any] = None, headers: Dict[str, str] = None):
    """Helper function to test any endpoint."""
    async with httpx.AsyncClient(base_url=BASE_URL) as client:
        try:
            if method.upper() == "GET":
                response = await client.get(endpoint, headers=headers)
            elif method.upper() == "POST":
                response = await client.post(endpoint, json=data, headers=headers)
            elif method.upper() == "PUT":
                response = await client.put(endpoint, json=data, headers=headers)
            else:
                return {"error": f"Unsupported method: {method}"}
            
            result = {
                "status_code": response.status_code,
                "success": 200 <= response.status_code < 300,
                "headers": dict(response.headers),
            }
            
            # Try to parse JSON response
            try:
                result["response"] = response.json()
            except:
                result["response"] = response.text
            
            return result
            
        except Exception as e:
            return {"error": str(e)}

print("✅ Helper function ready!")

In [16]:
# Test 1: Health Check
result = await test_endpoint("GET", "/auth/health")
print(f"Health Check - Status: {result.get('status_code')}")
print(f"Success: {result.get('success')}")
if result.get('response'):
    print(f"Response: {json.dumps(result['response'], indent=2)}")
if result.get('error'):
    print(f"Error: {result['error']}")

Health Check - Status: 200
Success: True
Response: {
  "message": "Authentication service is healthy",
  "success": true,
  "details": {
    "service": "auth",
    "status": "operational"
  }
}


In [17]:
# Test 2: Login with valid credentials
result = await test_endpoint("POST", "/auth/login", TEST_CREDENTIALS)
print(f"Login - Status: {result.get('status_code')}")
print(f"Success: {result.get('success')}")

# Extract tokens if successful
access_token = None
refresh_token = None
if result.get('success') and result.get('response'):
    response_data = result['response']
    if isinstance(response_data, dict) and 'data' in response_data:
        access_token = response_data['data'].get('access_token')
        refresh_token = response_data['data'].get('refresh_token')
        print(f"✅ Access token obtained: {access_token[:50]}..." if access_token else "❌ No access token")
        print(f"✅ Refresh token obtained: {refresh_token[:20]}..." if refresh_token else "❌ No refresh token")
else:
    print(f"Response: {json.dumps(result.get('response', {}), indent=2)}")

Login - Status: 200
Success: True
✅ Access token obtained: eyJhbGciOiJIUzI1NiIsImtpZCI6IjIxWEEwVlRWVHM2QnZaSm...
✅ Refresh token obtained: diqgtqosx7so...


In [18]:
# Test 3: Login with invalid credentials
invalid_creds = {"email": "test@example.com", "password": "wrongpassword"}
result = await test_endpoint("POST", "/auth/login", invalid_creds)
print(f"Invalid Login - Status: {result.get('status_code')}")
print(f"Expected 401: {result.get('status_code') == 401}")
if result.get('response'):
    print(f"Response: {json.dumps(result['response'], indent=2)}")

Invalid Login - Status: 401
Expected 401: True
Response: {
  "error": "http_error",
  "message": {
    "error": "Authentication failed",
    "error_code": "AUTH_ERROR"
  },
  "request_id": "unknown",
  "status_code": 401
}


In [19]:
# Test 4: Get current user info (requires access token)
if access_token:
    headers = {"Authorization": f"Bearer {access_token}"}
    result = await test_endpoint("GET", "/auth/me", headers=headers)
    print(f"Get User Info - Status: {result.get('status_code')}")
    print(f"Success: {result.get('success')}")
    if result.get('response'):
        user_data = result['response']
        if isinstance(user_data, dict) and 'user' in user_data:
            print(f"User ID: {user_data['user'].get('id')}")
            print(f"Email: {user_data['user'].get('email')}")
            print(f"Verified: {user_data['user'].get('is_verified')}")
else:
    print("❌ No access token available for authenticated tests")

Get User Info - Status: 200
Success: True
User ID: ff420e57-c248-4104-b778-182c9dd431fa
Email: krijajannis@gmail.com
Verified: True


In [20]:
# Test 5: Refresh token
if refresh_token:
    refresh_data = {"refresh_token": refresh_token}
    result = await test_endpoint("POST", "/auth/refresh", refresh_data)
    print(f"Token Refresh - Status: {result.get('status_code')}")
    print(f"Success: {result.get('success')}")
    if result.get('response'):
        print(f"New tokens received: {bool(result['response'].get('access_token'))}")
else:
    print("❌ No refresh token available for refresh test")

Token Refresh - Status: 200
Success: True
New tokens received: True


In [21]:
# Test 6: Password strength check
password_data = {"password": "TestPassword123!"}
result = await test_endpoint("POST", "/auth/check-password-strength", password_data)
print(f"Password Strength - Status: {result.get('status_code')}")
print(f"Success: {result.get('success')}")
if result.get('response'):
    strength_data = result['response']
    print(f"Strength: {strength_data.get('strength_label', 'Unknown')}")
    print(f"Score: {strength_data.get('score', 'N/A')}")
    print(f"Valid: {strength_data.get('is_valid', False)}")

Password Strength - Status: 200
Success: True
Strength: Very Weak
Score: 100
Valid: False


In [ ]:
# Debug: Check rate limiting configuration
result = await test_endpoint("GET", "/auth/debug/rate-limiting")
print(f"Rate Limiting Debug - Status: {result.get('status_code')}")
print(f"Success: {result.get('success')}")
if result.get('response'):
    debug_data = result['response']
    print(f"Rate limiting enabled: {debug_data.get('rate_limiting_enabled')}")
    print(f"Debug mode: {debug_data.get('debug_mode')}")
    print(f"Full response: {json.dumps(debug_data, indent=2)}")
else:
    print("No debug endpoint available - let's check health endpoint for debug info")
    
# Check health endpoint again for debug info
health_result = await test_endpoint("GET", "/auth/health")
if health_result.get('response'):
    print(f"\nHealth endpoint debug info: {health_result['response'].get('debug', 'Not available')}")

In [ ]:
# Test 7: Forgot password
forgot_data = {"email": "test@example.com"}
result = await test_endpoint("POST", "/auth/forgot-password", forgot_data)
print(f"Forgot Password - Status: {result.get('status_code')}")
print(f"Success: {result.get('success')}")
if result.get('response'):
    print(f"Message: {result['response'].get('message', 'No message')}")

## Quick cURL Commands

Use these for fast testing from terminal:

In [ ]:
# Health check
curl -s "http://dev.krija.info:8000/api/auth/health" | jq '.'

In [ ]:
# Login
curl -s -X POST "http://dev.krija.info:8000/api/auth/login" \
  -H "Content-Type: application/json" \
  -d '{"email": "krijajannis@gmail.com", "password": "221224"}' | jq '.'

In [ ]:
# Test invalid login
curl -s -X POST "http://dev.krija.info:8000/api/auth/login" \
  -H "Content-Type: application/json" \
  -d '{"email": "test@example.com", "password": "wrongpassword"}' | jq '.'

In [ ]:
# Test password strength
curl -s -X POST "http://dev.krija.info:8000/api/auth/check-password-strength" \
  -H "Content-Type: application/json" \
  -d '{"password": "TestPassword123!"}' | jq '.'

## Troubleshooting

### Rate Limiting Issues
If you get 429 errors:
1. Wait 15 minutes for rate limit to reset
2. Or restart the server to clear cache

### Check Configuration
Verify server settings:

In [ ]:
# Check if rate limiting should be disabled
cd /Users/jannis/Developer/Cookify/backend
python -c "from core.config import settings; print(f'Debug: {settings.debug}'); print(f'Rate limiting: {settings.rate_limiting_enabled_safe}')"

## Test Summary

### Available Endpoints:
- ✅ `/auth/health` - Health check
- ✅ `/auth/login` - User login
- ✅ `/auth/register` - User registration
- ✅ `/auth/refresh` - Token refresh
- ✅ `/auth/logout` - User logout
- ✅ `/auth/me` - Current user info
- ✅ `/auth/profile` - User profile
- ✅ `/auth/forgot-password` - Password reset request
- ✅ `/auth/reset-password` - Password reset confirm
- ✅ `/auth/verify-email` - Email verification
- ✅ `/auth/resend-verification` - Resend verification
- ✅ `/auth/change-password` - Change password
- ✅ `/auth/check-password-strength` - Password strength
- ✅ `/auth/user-info` - Optional user info
- ✅ `/auth/dev-login` - Development login

### Test Files Created:
- All 16 endpoint test files
- Manual testing script
- This interactive notebook
- Comprehensive pytest configuration

In [ ]:
import time
print("⏳ Waiting for rate limit to expire...")
time.sleep(35)  # Wait 35 seconds to be safe
print("✅ Rate limit should have expired, testing now...")

# Test health endpoint again
result = await test_endpoint("GET", "/auth/health")
print(f"Health Check - Status: {result.get('status_code')}")
print(f"Success: {result.get('success')}")
if result.get('response'):
    print(f"Response: {json.dumps(result['response'], indent=2)}")
else:
    print(f"Still rate limited: {result.get('error')}")

## Server Management

Since the API runs on a remote server (`dev.krija.info:8000`), we cannot restart it directly.

### Options to fix rate limiting:
1. **Wait for rate limit to expire** (15 minutes)
2. **Contact server administrator** to restart the service
3. **Check if there's a server restart endpoint**
4. **Modify rate limiting settings** on the server

### Current Status:
- Server: ✅ Running on dev.krija.info:8000
- Debug Mode: ✅ Enabled (`debug: true`)
- Rate Limiting: ❌ Still blocking requests (middleware issue)

### Root Cause:
The rate limiting middleware is not properly reading the `rate_limiting_enabled_safe` property that should disable rate limiting in debug mode.

In [ ]:
# Test if there are admin/dev endpoints to restart or check server status
print("🔍 Checking for admin/dev endpoints...")

# Try common development/admin endpoints
dev_endpoints = [
    "/dev/restart",
    "/admin/restart", 
    "/auth/dev/restart",
    "/auth/dev/status",
    "/dev/health",
    "/admin/health",
    "/system/restart",
    "/api/dev/restart"
]

for endpoint in dev_endpoints:
    try:
        result = await test_endpoint("POST", endpoint)
        if result.get('status_code') != 429:
            print(f"✅ Found endpoint: {endpoint} - Status: {result.get('status_code')}")
            if result.get('response'):
                print(f"Response: {json.dumps(result['response'], indent=2)}")
        else:
            print(f"❌ {endpoint} - Rate limited")
    except Exception as e:
        print(f"❌ {endpoint} - Error: {e}")
    
    # Small delay to avoid overwhelming
    await asyncio.sleep(0.5)

In [ ]:
# Try to get debug information about rate limiting
print("\n🔧 Checking debug endpoints...")

debug_endpoints = [
    "/auth/debug",
    "/debug/middleware",
    "/debug/rate-limit",
    "/auth/debug/middleware",
    "/auth/debug/config"
]

for endpoint in debug_endpoints:
    try:
        result = await test_endpoint("GET", endpoint)
        if result.get('status_code') != 429:
            print(f"✅ Debug endpoint found: {endpoint} - Status: {result.get('status_code')}")
            if result.get('response'):
                print(f"Response: {json.dumps(result['response'], indent=2)}")
                break
        else:
            print(f"❌ {endpoint} - Rate limited")
    except Exception as e:
        pass
    
    await asyncio.sleep(0.5)

## Next Steps for Server Administrator

Since the rate limiting middleware is not respecting the debug mode setting, the server administrator needs to:

### 1. Check Server Logs
Look for the debug logging we added to the middleware:
```bash
# Check server logs for rate limiting debug messages
tail -f /path/to/server/logs/app.log | grep "Rate limiting"
```

### 2. Restart the Server
```bash
# Restart the FastAPI server
pkill -f "python.*main.py"
python main.py
# or if using uvicorn directly:
uvicorn main:app --reload --host 0.0.0.0 --port 8000
```

### 3. Verify Configuration
```bash
# Check that the configuration is loaded correctly
python -c "from core.config import settings; print(f'Debug: {settings.debug}'); print(f'Rate limiting: {settings.rate_limiting_enabled_safe}')"
```

### 4. Alternative: Temporarily Disable Middleware
Comment out the rate limiting middleware in `main.py`:
```python
# application.add_middleware(AuthRateLimitMiddleware)  # Temporarily disabled
```

Once the server is restarted, the tests should work properly.